#  STM DOS Demo

This notebook demos a use case that involves finding a feasible parameterization for an STM simulation from an experimental image

In [ ]:
import os
import pySPM
import numpy as np
import matplotlib.pyplot as plt

from ingrained.image_ops import crop_by_ratio, score_vifp

# Main ingrained package modules
from ingrained.structure import PartialCharge
from ingrained.optimize import CongruityBuilderSTM

In [ ]:
# Set path for stm image
sxm_path = "sxm/0729LHe_D_BAgM14056.sxm"
# sxm_path = "sxm/0729LHe_D_BAgM14064.sxm"

# Set path for PARCHG file
pchg_path = "structures/PARCHG"
# pchg_path = "structures/PARCHG_4"

In [ ]:
# Read the image from file into an array.
img = pySPM.SXM(sxm_path).get_channel('Z', direction='both',corr="slope").pixels

# An optional way to view experimental image
plt.imshow(img, cmap='hot'); plt.axis('off');

There are several noisy regions in the sample image. Help the optimizer achieve better results by restricting the search to a relatively clean region of the sample. 

In [ ]:
#  For 0729LHe_D_BAgM14056.sxm
rrow = (0.10,0.65) # start row (ratio), end row (ratio)
rcol = (0.40,1.00) # start col (ratio), end col (ratio)  

# For 0729LHe_D_BAgM14064.sxm
# rrow = (0.20,0.95) # start row (ratio), end row (ratio)
# rcol = (0.30,1.00) # start col (ratio), end col (ratio)

# Crop the image
cropped_img = crop_by_ratio(img,rrow,rcol)

# View the cropped image to make sure it contains a reasonable portion of the initial image
plt.imshow(cropped_img, cmap='hot'); plt.axis('off');

In [ ]:
# Initialize a PartialCharge object from a PARCHG file
parchar = PartialCharge(pchg_path)

# Initialize a ConguityBuilder to optimize a the fit between a PARCHG structure and the experiment
congruity = CongruityBuilderSTM(partial_charge=parchar, \
                                sxm=sxm_path, \
                                restrict_bounds=[(rrow[0], rrow[1]),(rcol[0], rcol[1])])

Define a starting point for optimization.  <br/>**NOTE**: Picking good starting parameters is very important.
If you have no intuition about this, consider using a COBYLA optimizer to find reasonable starting point!
(Will take ~30 mins to run)

In [ ]:
# Define a starting point for optimization 
#    | zthk | ztol | rval | rtol | rang | pixs |
x0 = [-0.5 , 8.0 , 4 , 3.5 , 285 , 0.16]

In [ ]:
# Perform *unconstrained* minimization using a POWELL solver for further refine the solution from the previous step.
# NOTE: Takes sequential one-dimensional minimizations, 
#       so convergence can be quite slow. Violation of any HARD 
#       constraints defined inside the PartialCharge and
#       ConguityBuilderSTM classes will throw an arbitrarily 
#       large '9999' FOM value.

res = congruity.find_correspondence(optimizer='Powell', initial_solution=x0)

Get the parameter values from the result and print out a final 'ingrained' image with the corresponding simulated and experimental images.

In [ ]:
zthick,ztol,rho0,rho_tol,rotation_angle,pixel_size = res.x

# DO NOT FORGET TO DO THIS - these values were scaled up to ranges commensurate with other variables
rho0    = float(rho0)/1000
rho_tol = float(rho_tol)/1000

congruity.fit(zthick=zthick, \
              ztol=ztol, \
              rho0=rho0, \
              rho_tol=rho_tol, \
              pixel_size=pixel_size, \
              rotation_angle=rotation_angle, \
              save_simulation = 'results/'+sxm_path.split("/")[-1]+"_sim.jpg", \
              save_experiment = 'results/'+sxm_path.split("/")[-1]+"_exp.jpg", \
              display=True)